In [7]:
from urllib.request import urlopen

In [8]:
import cv2


In [9]:
import numpy as np

In [10]:
url="http://100.78.214.196:8080/shot.jpg?rnd=147366"

In [11]:
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [12]:
import cv2
f1=open('face/count.txt')
f=f1.readline()
w=f.split()
l1=w[:1]
c=[int(x) for x in l1]
count=c[0]+10
count_str=str(count)
file=open("face/count.txt",mode="w",encoding="utf-8")
file.write(count_str)
file.close()


#getting last FACE ID from file and updating

f1=open('face/id_count.txt')
f=f1.readline()
w=f.split()
l1=w[:1]
face_ID=[int(x) for x in l1]
print(face_ID)
face_id=face_ID[0]+1
f_id=str(face_id)
file=open("face/id_count.txt",mode="w",encoding="utf-8")
file.write(f_id)
file.close()


import csv  
fields=[f_id,'ashad']
with open(r'face/id_and_face_info.csv', 'a') as f:
    writer = csv.writer(f)
    writer.writerow(fields)

    
    
while 1:
    imgResp=urlopen(url)
    imgNp=np.array(bytearray(imgResp.read()),dtype=np.uint8)
    img=cv2.imdecode(imgNp,-1)
    img=cv2.resize(img,(600,400))
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_cascade.detectMultiScale(gray,1.3,5)
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,255,0),2)  
        count += 1
        # Save the captured image into the datasets folder
        cv2.imwrite("face/dataset/User." + str(face_id) + '.' +   str(count) + ".jpg", gray[y:y+h,x:x+w])
        cv2.imshow('image', img)
    k = cv2.waitKey(100) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break
    elif count >= 10: # Take 10 face sample and stop video
         break# Do a bit of cleanup       
print("\n [INFO] Exiting Program and cleanup stuff")
cv2.destroyAllWindows()


[104]

 [INFO] Exiting Program and cleanup stuff


In [12]:
import cv2
import numpy as np
from PIL import Image
import os# Path for face image database
path = 'face/dataset'
recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml");# function to get the images and label data
def getImagesAndLabels(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    ids = []
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L') # grayscale
        img_numpy = np.array(PIL_img,'uint8')
        id = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = detector.detectMultiScale(img_numpy)
        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            ids.append(id)
    return faceSamples,ids
print ("\n [INFO] Training faces. It will take a few seconds. Wait ...")
faces,ids = getImagesAndLabels(path)
recognizer.train(faces, np.array(ids))
# Save the model into trainer/trainer.yml
recognizer.write('face/trainer.yml') 
# Print the numer of faces trained and end program
print("\n [INFO] {0} faces trained. Exiting Program".format(len(np.unique(ids))))


 [INFO] Training faces. It will take a few seconds. Wait ...

 [INFO] 2 faces trained. Exiting Program


In [15]:
import cv2
import numpy as np
import os 

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('face/trainer.yml')
cascadePath = "haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascadePath);
font = cv2.FONT_HERSHEY_SIMPLEX#iniciate id counter

imgResp=urlopen(url)
imgNp=np.array(bytearray(imgResp.read()),dtype=np.uint8)
img=cv2.imdecode(imgNp,-1)
img=cv2.resize(img,(600,400))
gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
faces=face_cascade.detectMultiScale(gray,1.3,5)
for(x,y,w,h) in faces:
    cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)
    id, confidence = recognizer.predict(gray[y:y+h,x:x+w])
            # If confidence is less them 100 ==> "0" : perfect match 
    if (confidence < 100):
        id = id
        print(type(id))
        confidence = "  {0}%".format(round(100 - confidence))
    else:
        id = "unknown"
        print(id)
        confidence = "  {0}%".format(round(100 - confidence))
print("\n [INFO] Exiting Program and cleanup stuff")
cv2.destroyAllWindows()




 [INFO] Exiting Program and cleanup stuff
